In [9]:
import os
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

In [10]:
# ===============================
# Test Dataset
# ===============================
class TestDataset(Dataset):
    def __init__(self, test_dir, csv_path=None):
        self.test_dir = test_dir
        
        if csv_path is not None:
            # Validation mode (with labels)
            self.data = pd.read_csv(csv_path, dtype={'ID': str})
            self.has_labels = True
        else:
            # Submission mode (no labels)
            img_files = [f for f in os.listdir(test_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            self.data = pd.DataFrame({'ID': [os.path.splitext(f)[0] for f in img_files]})
            self.has_labels = False

        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_id = str(self.data.iloc[idx]['ID'])
        img_path = os.path.join(self.test_dir, img_id + ".jpg")
        
        image = Image.open(img_path).convert("RGB")
        image = self.transform(image)

        if self.has_labels:
            label = int(self.data.iloc[idx]['label'])
            return image, label, img_id
        else:
            return image, img_id

In [11]:
# ===============================
# CNN Model
# ===============================
import timm

class CNN(nn.Module):  # ← Keep name CNN() so TA's code doesn't break!
    def __init__(self, pretrained=True, freeze_backbone=True, dropout=0.3):
        super(CNN, self).__init__()

        # ViT-Base
        self.vit = timm.create_model('vit_base_patch16_224', pretrained=pretrained, num_classes=0)  # 768-dim
        # Swin-Base
        self.swin = timm.create_model('swin_base_patch4_window7_224', pretrained=pretrained, num_classes=0)  # 1024-dim

        # Freeze backbones (highly recommended for AIGC detection)
        if freeze_backbone:
            for p in self.vit.parameters():
                p.requires_grad = False
            for p in self.swin.parameters():
                p.requires_grad = False

        # Fusion head
        self.fusion = nn.Sequential(
            nn.Linear(768 + 1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),

            nn.Linear(512, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),

            nn.Linear(128, 2)  # ← 2 classes: real vs AIGC
        )

    def forward(self, x):
        vit_feat = self.vit(x)
        swin_feat = self.swin(x)

        if swin_feat.dim() == 3:
            swin_feat = swin_feat.mean(1)

        combined = torch.cat([vit_feat, swin_feat], dim=1)
        out = self.fusion(combined)
        return out

In [12]:
def test_model():
    data_root = "./"
    model_path = "model.pth"

    test_dir = os.path.join(data_root, "test")
    output_csv = "submission.csv" 

    print(f"Loading images from: {test_dir}")
    print(f"Loading model from: {model_path}")

    # Load Dataset
    test_dataset = TestDataset(test_dir, csv_path=None)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0, pin_memory=True)

    # Load model
    model = CNN(pretrained=False, freeze_backbone=True).to(device)
    # Load trained weights from the model
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    print("Model loaded")

    results = []

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Predicting"):
            if len(batch) == 3:
                imgs, _, img_ids = batch
            else:
                imgs, img_ids = batch

            imgs = imgs.to(device)
            outputs = model(imgs)
            preds = outputs.argmax(dim=1).cpu().numpy()  # 0 or 1

            for img_id, pred in zip(img_ids, preds):
                results.append({"ID": img_id, "label": int(pred)})

    # Save to CSV
    submission_df = pd.DataFrame(results)
    submission_df.to_csv(output_csv, index=False)
    print(f"\nSubmission saved to '{output_csv}'")
    print(f"Total predictions: {len(submission_df)}")
    print(submission_df.head())

# Run it
if __name__ == "__main__":
    test_model()

Loading images from: ./test
Loading model from: model.pth


/var/folders/j_/v7yb81ws07d2vjpxh5tjlzqm0000gn/T/ipykernel_45268/932493408.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, 

Model loaded successfully!


Predicting: 100%|██████████| 79/79 [03:54<00:00,  2.97s/it]


Submission saved to 'submission.csv'
Total predictions: 2500
              ID  label
0  image_0002064      1
1  image_0000673      1
2  image_0000115      0
3  image_0000101      0
4  image_0001579      1


In [13]:
data_root = "path_to_dataset"
model_path = "saved_model.pth"